In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.7 MB/s eta 0:00:00


# Load Perseus

In [ ]:
import pandas as pd
semecor = True
if semecor:
  # do not use perseus, but actually semcor
  perseus = pd.read_table("semcor_translated.tsv", header=None)
else:
  perseus = pd.read_csv("text-alignment.csv")

perseus.columns = ["Latin", "English"]

In [ ]:
perseus.head()

,Latin,English
0,Quousque est quia non recognoscitis in aliquip...,How long has it been since you reviewed the ob...
1,Quousque est quia non recognoscitis in aliquip...,How long has it been since you reviewed the ob...
2,Have vos permittitur ut sit a giveaway progres...,Have you permitted it to become a giveaway pro...
3,Quid conatus facis ad assess eventus vestri pr...,What effort do you make to assess results of y...
4,"Tibi mensuram suam reducitur absenteeism, turn...",Do you measure its relation to reduced absente...


In [ ]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, perseus):
      self.corpus = perseus.Latin

    def __len__(self):
        return len(self.corpus)

    def __getitem__(self, i):
        return self.corpus[i]

dataset = MyDataset(perseus)

In [ ]:
def data(perseus):
    for i in range(len(perseus)):
        yield perseus.Latin[i]

# POS

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from tqdm.auto import tqdm

In [ ]:
pipe = pipeline(model="enelpol/evalatin2022-pos-open", aggregation_strategy="max", device=0, batch_size=16)

In [ ]:
POS_sentences = []
for out in tqdm(pipe(dataset, batch_size=32), total=len(dataset)):
        POS_sentences.append(["$".join([w["word"], w["entity_group"]]) for w in out])

  0%|          | 0/37176 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


KeyError: ignored

In [ ]:
POS_sentences[0]

['Quousque$ADV',
 'est$AUX',
 'quia$SCONJ',
 'non$PART',
 'recognoscitis$VERB',
 'in$ADP',
 'aliquip$ADJ',
 'tui$PRON',
 'beneficium$NOUN',
 'et$CCONJ',
 'servitiumprogressio?$NOUN']

In [ ]:
import pickle

with open("POS_annotated_semcor.pkl", "wb") as f:
  pickle.dump(POS_sentences, f)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle

with open("POS_annotated_perseus.pkl", "rb") as f:
  POS_sentences = pickle.load(f)

In [ ]:
import re

def correct_pos_results(pos):
  new_pos = []
  for idx in range(len(pos)):
    sent = pos[idx]
    new_sent = []
    for word_idx, word in enumerate(sent):
      if word=="$X":
        continue
      lemma = word.split("$")[0].split()
      try:
        check = len(re.sub("\W", "", lemma[0]))
      except IndexError:
        print(word_idx)
        print(idx)
        print(lemma)
        0/0
      if check==1:
        try:
          new_sent[-1]="$".join([new_sent[-1].split("$")[0]+lemma[0],new_sent[-1].split("$")[1]])
        except IndexError:
          print(new_sent)
          print(word)
          print(lemma)
          0/0
        if len(lemma)>1:
          new_sent.append("$".join([lemma[1], word.split("$")[1]]))
      else:
        new_sent.append(word)
    new_pos.append(sent)
  return new_pos

def check_pos_perseus(pos, perseus, all_attached=False):
  error_idx=[]
  for idx, p in enumerate(pos):
    perseus_sent = "".join(perseus.Latin[idx].split()) if all_attached else perseus.Latin[idx]
    if all_attached:
      words = []
      for w in p:
        try:
          words.extend(w.split("$")[0].split())
        except IndexError:
          pass
      sent = "".join(words)
    else:
      sent = " ".join([w.split("$")[0] for w in p])
    if sent!=perseus_sent:
      perseus_sent = re.sub('�', "", perseus_sent)
      if sent!=perseus_sent:
        error_idx.append(idx)
  return error_idx

In [ ]:
errors = check_pos_perseus(POS_sentences, perseus, True)

In [ ]:
errors

[85,
 86,
 407,
 422,
 423,
 699,
 700,
 807,
 808,
 823,
 824,
 848,
 849,
 956,
 1408,
 1462,
 1463,
 1741,
 1742,
 1744,
 1745,
 1772,
 1776,
 1777,
 1785,
 1786,
 2337,
 2338,
 2366,
 2367,
 2646,
 2647,
 2801,
 2802,
 2862,
 2863,
 2885,
 3473,
 3474,
 3475,
 3492,
 3493,
 3511,
 3512,
 3520,
 3523,
 3524,
 3531,
 3532,
 3774,
 3775,
 3776,
 3777,
 3778,
 3779,
 3780,
 3790,
 3791,
 3792,
 3855,
 3856,
 4541,
 4542,
 4584,
 4585,
 4655,
 4656,
 4847,
 4848,
 4866,
 4872,
 4873,
 4902,
 4903,
 5019,
 5020,
 5021,
 5022,
 5025,
 5026,
 5040,
 5043,
 5044,
 5045,
 5058,
 5059,
 5174,
 5358,
 5366,
 5367,
 5812,
 5813,
 6249,
 6250,
 6255,
 6256,
 6259,
 6260,
 6262,
 6263,
 6268,
 6269,
 6271,
 6481,
 6482,
 6492,
 6493,
 6528,
 6529,
 6530,
 6531,
 6532,
 6533,
 6534,
 6535,
 6536,
 6765,
 6838,
 6839,
 6843,
 6844,
 6845,
 6846,
 6847,
 6848,
 6937,
 7473,
 7474,
 7745,
 7946,
 7948,
 7949,
 8432,
 8433,
 8434,
 8486,
 8487,
 8763,
 8774,
 8777,
 8778,
 8779,
 8780,
 8887,
 8957,
 

# Lemma

In [ ]:
search_lemmas = set(['acerbus$ADJ', 'adsumo$VERB', 'ancilla$NOUN', 'beatus$ADJ',
       'civitas$NOUN', 'cohors$NOUN', 'consilium$NOUN', 'consul$NOUN',
       'credo$VERB', 'dolus$NOUN', 'dubius$ADJ', 'dux$NOUN',
       'fidelis$ADJ', 'fidelis$NOUN', 'honor$NOUN', 'hostis$NOUN',
       'humanitas$NOUN', 'imperator$NOUN', 'itero$VERB', 'ius$NOUN',
       'licet$VERB', 'necessarius$ADJ', 'nepos$NOUN', 'nobilitas$NOUN',
       'oportet$NOUN', 'poena$NOUN', 'pontifex$NOUN', 'potestas$NOUN',
       'regnum$NOUN', 'sacramentum$NOUN', 'salus$NOUN', 'sanctus$ADJ',
       'sapientia$NOUN', 'scriptura$NOUN', 'senatus$NOUN', 'sensus$NOUN',
       'simplex$ADJ', 'templum$NOUN', 'titulus$NOUN', 'virtus$NOUN',
       'voluntas$NOUN'])

In [ ]:
pipe = pipeline("text2text-generation", model="enelpol/evalatin2022-lemma-open", device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import re

def transform_to_lemma_input(POS_sentences):
  new_sentences = []
  new_POS_sentences = []
  for sent in POS_sentences:
    new_sent = []
    new_POS = []
    for token in sent:
      try:
        lemma, pos = token.split("$")
      except ValueError:
        pass
      if len(lemma.split())>1:
          split_lemma = lemma.split()
          new_sent.extend([f":{l}:{pos}" for l in split_lemma])
          new_POS.extend(["$".join([l,pos]) for l in split_lemma])
      else:
        lemma = re.sub("\W", "", lemma)
        if len(lemma)>2:
          new_sent.append(f":{lemma}:{pos}")
          new_POS.append(token)
    new_sentences.append(new_sent)
    new_POS_sentences.append(new_POS)
  return new_sentences, new_POS_sentences

In [ ]:
lemma_input, new_POS_sentences = transform_to_lemma_input(POS_sentences)

In [ ]:
results = {"Latin":[], "English":[], "Lemma":[], "PoS":[], "target_word":[]}

In [ ]:
def search_for_lemma(search_lemmas, lemma_pos):
  lemma = lemma_pos.split("$")[1]
  if lemma in search_lemmas:
    return lemma
  else:
    return False

In [ ]:
for doc_idx, sample in tqdm(enumerate(lemma_input), total=len(lemma_input)):
  if not sample:
    continue
  bs = len(sample)
  lemmas = pipe(sample, batch_size=bs)
  for idx, l in enumerate(lemmas):
    try:
      lemma, pos = new_POS_sentences[doc_idx][idx].split("$")
    except ValueError:
      continue
    lemma_pos = "$".join([l["generated_text"], pos])
    if lemma_pos in search_lemmas:
      results["Latin"].append(perseus.Latin[doc_idx])
      results["English"].append(perseus.English[doc_idx])
      results["Lemma"].append(l["generated_text"])
      results["PoS"].append(pos)
      results["target_word"].append(lemma)

  0%|          | 0/37152 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize ef

In [ ]:
df = pd.DataFrame(results)
df.to_csv("all_perseus_matching.csv", index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pd.unique(df.Lemma)

array(['necessarius', 'ius', 'sensus', 'dubius', 'nobilitas', 'potestas',
       'consilium', 'sapientia', 'regnum', 'poena', 'credo', 'salus',
       'titulus', 'nepos', 'beatus', 'ancilla', 'acerbus', 'consul',
       'honor', 'dolus', 'pontifex'], dtype=object)

In [ ]:
stats = {"Lemma":[], "PoS":[], "frequency":[]}

In [ ]:
for lemma_pos in pd.unique(df[["Lemma", "PoS"]].apply("$".join, axis = 1)):
  lemma, pos = lemma_pos.split("$")
  subset = df[(df.Lemma==lemma)&(df.PoS==pos)]

  stats["Lemma"].append(lemma)
  stats["PoS"].append(pos)
  stats["frequency"].append(len(subset))

In [ ]:
pd.DataFrame(stats)

,Lemma,PoS,frequency
0,ius,NOUN,1618
1,licet,VERB,1181
2,scriptura,NOUN,85
3,potestas,NOUN,770
4,fidelis,ADJ,233
5,credo,VERB,2104
6,consilium,NOUN,1924
7,dubius,ADJ,299
8,salus,NOUN,1040
9,ancilla,NOUN,194


In [ ]:
stats = pd.DataFrame(stats).to_csv("lemma_stats.csv", index=False)